In [ ]:
#To change the working directory
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Applied-Semester-Project')
!pwd

/content/drive/My Drive/Colab Notebooks/Applied-Semester-Project


In [ ]:
#To install all libraries needed 
!pip install selenium
!pip install kora -q

     |████████████████████████████████| 61kB 2.5MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 


In [ ]:
#Import all libraries needed 
from kora.selenium import wd
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
from pandas import read_excel
import csv

In [ ]:
# Step 1: Login to LinkedIn
  #This file containt my Linkedin account 
credential = open("account.txt")
line = credential.readlines()
username = line[0]
password = line[1]
# Open Chrome and login LinkedIn login site
  # Open Chrome and login 
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

  #Fill in username and password
email_field = driver.find_element_by_id("username")
email_field.send_keys(username)
sleep(2)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
sleep(2)

  #Login 
login_field = driver.find_element_by_xpath('//*[@id="app__container"]/main/div[2]/form/div[3]/button')
login_field.click()
sleep(3)


In [ ]:
# Step 2: Search for the profile we want to crawl
    #Locate the search bar
search_field = driver.find_element_by_xpath('//*[@id="ember20"]/input')
    #Input the search querry to the search bar
search_querry = input("What profile you want to search?")
search_field.send_keys(search_querry)
    #Search
search_field.send_keys(Keys.RETURN)

    #Click see all people
see_all_field = driver.find_element_by_xpath('/html/body/div[7]/div[3]/div/div[2]/div/div[2]/div/div[1]/div/a')
see_all_field.click()

In [ ]:
# Step 3: Scrape the urls of a profile
def GetURLs_on_Pages():
    number_of_page = int(input("How many pages you want to scrape? "))
    URLs_all_page = []
    for page in range(number_of_page):
        URLs_one_page = Get_URL()
        sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        sleep(2)
        next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')
        next_button.click()
        URLs_all_page += URLs_one_page
        sleep(2)
    return URLs_all_page

URLs_all_page = GetURLs_on_Pages()
URLs_all_page

In [ ]:
# Step 3: Scrape the urls of the profiles ans save to a CSV
with open('Linkedin_URLs.txt','w') as f:
    number_of_page = int(input("How many pages you want to scrape? "))
    for page in range(number_of_page):
        URLs_one_page = Get_URL()
        sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        sleep(2)
        next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')
        next_button.click()
        f.write(URLs_one_page + '\n')
        sleep(2)

In [ ]:
#Read urls from csv file
with open('Canada.csv', 'r') as f:
    URLs_all_page = f.read().split("\n")
for linkedin_URL in URLs_all_page:
    print(linkedin_URL)

In [ ]:
# Step 4: Scrape the info of the profiles ans save to a CSV file

with open('output.csv', 'w', newline = '', encoding='utf8') as f:
    headers = ['Name', 'Job Title', 'Location', 'Skills', 'URL']
    writer = csv.DictWriter(f, delimiter=',', lineterminator='\n', fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        sleep(2)
        show_more_button = driver.find_element_by_class_name('pv-skills-section__additional-skills')
        show_more_button.click()
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div', class_ = "flex-1 mr5")
        info_loc = info_div.find_all('ul')
        name = info_loc[0].find('li').get_text().strip()
        location = info_loc[1].find('li').get_text().strip()
        title = info_div.find('h2').get_text().strip()
        info_skill = page_source.find_all('span', class_ = "pv-skill-category-entity__name-text t-16 t-black t-bold")
        skills = []
        for skill in info_skill:
            skills.append(skill.get_text().strip())
        skills = ", ".join(skills)
        writer.writerow({headers[0]:name,
                    headers[1]:title,
                    headers[2]:location,
                    headers[3]:skills,
                    headers[4]:linkedin_URL,
                    })    
